In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
print('ddd')

ddd


In [18]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table
import psycopg2

In [19]:
# подгружаем .env
load_dotenv()

True

In [20]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')


In [21]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [22]:
import os

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

TABLE_NAME = 'clean_users_churn'

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000



In [23]:
import os

import psycopg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(5)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,No,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0
2,3,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,...,No,No,No,No,Male,0,No,No,No,1
3,4,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,...,Yes,Yes,No,No,Male,0,No,No,No,0
4,5,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,...,No,No,No,No,Female,0,No,No,No,1


In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from autofeat import AutoFeatRegressor,AutoFeatClassifier
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
import os
import mlflow
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss


In [25]:
features = [] # список признаков вашей модели
target = ['target'] # колонка с таргетом вашей модели
cat_features = [
    'paperless_billing',
    'payment_method',
    'internet_service',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    'gender',
    'senior_citizen',
    'partner',
    'dependents',
    'multiple_lines',
]
num_features = ["monthly_charges", "total_charges"]

features = cat_features + num_features
split_column = "begin_date"
test_size = 0.2

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],  # теперь df[target] не пустой
    test_size=test_size,
    shuffle=False,
)

In [27]:
import os
import numpy as np
import optuna
from optuna.integration.mlflow import MLflowCallback
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss
)
from collections import defaultdict
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
import mlflow

# Установка переменных окружения
os.environ.update({
    "MLFLOW_S3_ENDPOINT_URL": "https://storage.yandexcloud.net",
    "AWS_ACCESS_KEY_ID": os.getenv("AWS_ACCESS_KEY_ID"),
    "AWS_SECRET_ACCESS_KEY": os.getenv("AWS_SECRET_ACCESS_KEY"),
})


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# Параметры эксперимента
EXPERIMENT_NAME = 'churn_denis_putov_www'
RUN_NAME = "model_bayesian_search_@2222"
STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

# Загрузка и подготовка данных
features = cat_features + num_features
target = 'target'

df = df.sort_values(by=[split_column])
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=test_size,
    shuffle=False,
)

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
        "cat_features": cat_features
    }
    
    model = CatBoostClassifier(**param)
    skf = StratifiedKFold(n_splits=2)
    
    metrics = defaultdict(list)
    
    for train_index, val_index in skf.split(X_train, y_train):
        train_X = X_train.iloc[train_index]
        train_y = y_train.iloc[train_index]
        val_X = X_train.iloc[val_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(
            train_X,
            train_y,
            eval_set=(val_X, val_y),
            early_stopping_rounds=20,
            verbose=False
        )
        
        probas = model.predict_proba(val_X)[:, 1]
        prediction = model.predict(val_X)
        
        tn, fp, fn, tp = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        metrics["err1"].append(fp)
        metrics["err2"].append(fn)
        metrics["auc"].append(roc_auc_score(val_y, probas))
        metrics["precision"].append(precision_score(val_y, prediction))
        metrics["recall"].append(recall_score(val_y, prediction))
        metrics["f1"].append(f1_score(val_y, prediction))
        metrics["logloss"].append(log_loss(val_y, probas))
    
    # Агрегация метрик
    err1 = np.mean(metrics["err1"])
    err2 = np.mean(metrics["err2"])
    auc = np.mean(metrics["auc"])
    precision = np.mean(metrics["precision"])
    recall = np.mean(metrics["recall"])
    f1 = np.mean(metrics["f1"])
    logloss = np.mean(metrics["logloss"])
    
    # Сохранение метрик
    trial.set_user_attr("err1", err1)
    trial.set_user_attr("err2", err2)
    trial.set_user_attr("precision", precision)
    trial.set_user_attr("recall", recall)
    trial.set_user_attr("f1", f1)
    trial.set_user_attr("logloss", logloss)
    
    return auc

# Создание/получение эксперимента
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

# Запуск оптимизации
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflc = MLflowCallback(
    tracking_uri=f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}",
    metric_name="auc",
    create_experiment=False,
    mlflow_kwargs={
    'experiment_id': experiment_id,
    'tags': {MLFLOW_PARENT_RUN_ID: run_id}
    }
    )

    study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(),
    direction="maximize",
    study_name=STUDY_NAME,
    storage=STUDY_DB_NAME,
    load_if_exists=True
    )

    study.optimize(
    objective,
    n_trials=10,
    callbacks=[mlflc],
    show_progress_bar=True
    )

    # Логирование лучшей модели
    best_params = study.best_params
    best_model = CatBoostClassifier(**best_params, cat_features=cat_features)
    best_model.fit(X_train, y_train, verbose=False)

    mlflow.catboost.log_model(
    cb_model=best_model,
    artifact_path="best_model",
    registered_model_name="best_churn_model"
    )

    # Логирование параметров и метрик
    mlflow.log_params(best_params)
    mlflow.log_metrics({
    "auc": study.best_value,
    "err1": study.best_trial.user_attrs["err1"],
    "err2": study.best_trial.user_attrs["err2"],
    "precision": study.best_trial.user_attrs["precision"],
    "recall": study.best_trial.user_attrs["recall"],
    "f1": study.best_trial.user_attrs["f1"],
    "logloss": study.best_trial.user_attrs["logloss"]
    })

    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Best AUC: {study.best_value:.4f}")
    print("Best parameters:")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")

/tmp/ipykernel_292372/1560367384.py:123: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-04-06 17:09:02,797] Using an existing study with name 'churn_model' instead of creating a new one.


  0%|          | 0/10 [00:00<?, ?it/s]

/home/mle-user/mle_projects/.venv_name/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[I 2025-04-06 17:09:04,334] Trial 5 finished with value: 0.7812184846916026 and parameters: {'learning_rate': 0.02449856380183102, 'depth': 3, 'l2_leaf_reg': 0.3913113373531869, 'random_strength': 1.726084195631594}. Best is trial 5 with value: 0.7812184846916026.


Exception: Run with UUID 3d3b42216a2b4e9dbe218128810894ba is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [36]:
# Добавляем недостающие импорты в начале файла
import json
from datetime import datetime


In [ ]:
import os
from mlflow.models.signature import infer_signature

import numpy as np
import optuna
from optuna.integration.mlflow import MLflowCallback
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss
)
from collections import defaultdict
import mlflow
import logging

# Настройка логов
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Проверка переменных окружения
TRACKING_SERVER_HOST = os.getenv("TRACKING_SERVER_HOST", "localhost")
TRACKING_SERVER_PORT = os.getenv("TRACKING_SERVER_PORT", "5000")

if not all([TRACKING_SERVER_HOST, TRACKING_SERVER_PORT]):
    raise ValueError("Не заданы TRACKING_SERVER_HOST и TRACKING_SERVER_PORT!")

# Настройка MLflow
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)

logger.info(f"Tracking URI: {mlflow.get_tracking_uri()}")

# Параметры эксперимента
EXPERIMENT_NAME = 'churn_denis_putov_www'
RUN_NAME = "model_bayesian_search_@2222"
STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
        "cat_features": cat_features
    }
    
    model = CatBoostClassifier(**param)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=0)
    
    metrics = defaultdict(list)
    
    for train_index, val_index in skf.split(X_train, y_train):
        train_X = X_train.iloc[train_index]
        train_y = y_train.iloc[train_index]
        val_X = X_train.iloc[val_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(
            train_X,
            train_y,
            eval_set=(val_X, val_y),
            early_stopping_rounds=20,
            verbose=False
        )
        
        probas = model.predict_proba(val_X)[:, 1]
        prediction = model.predict(val_X)
        
        tn, fp, fn, tp = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        metrics["err1"].append(fp)
        metrics["err2"].append(fn)
        metrics["auc"].append(roc_auc_score(val_y, probas))
        metrics["precision"].append(precision_score(val_y, prediction, zero_division=0))
        metrics["recall"].append(recall_score(val_y, prediction, zero_division=0))
        metrics["f1"].append(f1_score(val_y, prediction, zero_division=0))
        metrics["logloss"].append(log_loss(val_y, probas))
    
    # Агрегация метрик
    trial.set_user_attr("err1", np.mean(metrics["err1"]))
    trial.set_user_attr("err2", np.mean(metrics["err2"]))
    trial.set_user_attr("precision", np.mean(metrics["precision"]))
    trial.set_user_attr("recall", np.mean(metrics["recall"]))
    trial.set_user_attr("f1", np.mean(metrics["f1"]))
    trial.set_user_attr("logloss", np.mean(metrics["logloss"]))
    
    return np.mean(metrics["auc"])

# Основной блок выполнения
# Основной блок выполнения
# Основной блок выполнения
# Основной блок выполнения
if __name__ == "__main__":
    # Инициализация MLflow callback и исследования Optuna
    mlflc = MLflowCallback(
        tracking_uri=mlflow.get_tracking_uri(),
        metric_name="auc",
        create_experiment=True,
        mlflow_kwargs={
            'experiment_name': EXPERIMENT_NAME,
            'tags': {"project": "customer_churn"}
        }
    )
    
    study = optuna.create_study(
        sampler=optuna.samplers.TPESampler(),
        direction="maximize",
        study_name=STUDY_NAME,
        storage=STUDY_DB_NAME,
        load_if_exists=True
    )
    
    # Запуск оптимизации ДО логирования модели
    study.optimize(
        objective,
        n_trials=10,
        callbacks=[mlflc],
        show_progress_bar=True
    )

    # Создаем родительский запуск для лучшей модели
    with mlflow.start_run(run_name="best_model") as model_run:
        best_params = study.best_params
        best_model = CatBoostClassifier(**best_params, cat_features=cat_features)
        best_model.fit(X_train, y_train, verbose=False)
        
        # Логируем параметры и метрики
        mlflow.log_params(best_params)
        mlflow.log_metrics({
            "auc": study.best_value,
            "err1": study.best_trial.user_attrs["err1"],
            "err2": study.best_trial.user_attrs["err2"],
            "precision": study.best_trial.user_attrs["precision"],
            "recall": study.best_trial.user_attrs["recall"],
            "f1": study.best_trial.user_attrs["f1"],
            "logloss": study.best_trial.user_attrs["logloss"]
        })
        
        # Логирование модели с исправленными параметрами
        model_info = mlflow.catboost.log_model(
            cb_model=best_model,
            artifact_path="model",
            registered_model_name="best_churn_model",
            input_example=X_train.iloc[:1].to_dict('records')[0],
            signature=infer_signature(X_train, y_train.astype(float)),  # Исправление для схемы
            extra_files={
                "model_metadata.json": json.dumps({
                    "features": features,
                    "cat_features": cat_features,
                    "timestamp": datetime.now().isoformat()
                })
            }
        )
        
        # Дополнительное сохранение модели
        model_dir = mlflow.get_artifact_uri("model")
        best_model.save_model(os.path.join(model_dir, "model.cbm"))

2025-04-06 17:37:03,025 INFO: Tracking URI: http://localhost:5000
/tmp/ipykernel_292372/3747235735.py:107: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-04-06 17:37:03,090] Using an existing study with name 'churn_model' instead of creating a new one.


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-04-06 17:37:07,158] Trial 47 finished with value: 0.8334970508404581 and parameters: {'learning_rate': 0.05140620802113343, 'depth': 1, 'l2_leaf_reg': 1.2069737576510196, 'random_strength': 1.9330733132696487}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 17:37:10,650] Trial 48 finished with value: 0.8405458189566171 and parameters: {'learning_rate': 0.07874294537781834, 'depth': 2, 'l2_leaf_reg': 0.9425709021546527, 'random_strength': 2.600076273845518}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 17:37:14,521] Trial 49 finished with value: 0.8406311066754848 and parameters: {'learning_rate': 0.04036878782343144, 'depth': 3, 'l2_leaf_reg': 0.7678655097530994, 'random_strength': 2.8281276259835755}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 17:37:17,441] Trial 50 finished with value: 0.8389984827763697 and parameters: {'learning_rate': 0.08196698676867159, 'depth': 6, 'l2_leaf_reg': 1.4116824984136644, 'random_strength':

/home/mle-user/mle_projects/.venv_name/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None


TypeError: CatBoost.save_model() got an unexpected keyword argument 'extra_files'

FileNotFoundError: [Errno 2] No such file or directory: 'your_data.csv'

In [ ]:
         select *
            from tags
            join runs on runs.run_uuid = '403a6c9521a04b9db76588e60b26c605'
                and runs.lifecycle_stage = 'active'
                and runs.status = 'FINISHED'
                and tags.value = '403a6c9521a04b9db76588e60b26c605'
                and tags.key = 'mlflow.parentRunId'

In [41]:
import os
import numpy as np
import optuna
from optuna.integration.mlflow import MLflowCallback
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss
)
from collections import defaultdict
import mlflow
import logging

# Настройка логов
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Проверка переменных окружени

# Настройка MLflow
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)
pip_requirements = "../requirements.txt"
logger.info(f"Tracking URI: {mlflow.get_tracking_uri()}")
input_example = X_test[:10]
# Параметры эксперимента
EXPERIMENT_NAME = 'churn_denis_putov_www'
RUN_NAME = "model_bayesian_search_@2222"
STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
        "cat_features": cat_features
    }
    
    model = CatBoostClassifier(**param)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=0)
    
    metrics = defaultdict(list)
    
    for train_index, val_index in skf.split(X_train, y_train):
        train_X = X_train.iloc[train_index]
        train_y = y_train.iloc[train_index]
        val_X = X_train.iloc[val_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(
            train_X,
            train_y,
            eval_set=(val_X, val_y),
            early_stopping_rounds=20,
            verbose=False
        )
        
        probas = model.predict_proba(val_X)[:, 1]
        prediction = model.predict(val_X)
        
        tn, fp, fn, tp = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        metrics["err1"].append(fp)
        metrics["err2"].append(fn)
        metrics["auc"].append(roc_auc_score(val_y, probas))
        metrics["precision"].append(precision_score(val_y, prediction, zero_division=0))
        metrics["recall"].append(recall_score(val_y, prediction, zero_division=0))
        metrics["f1"].append(f1_score(val_y, prediction, zero_division=0))
        metrics["logloss"].append(log_loss(val_y, probas))
    
    # Агрегация метрик
    trial.set_user_attr("err1", np.mean(metrics["err1"]))
    trial.set_user_attr("err2", np.mean(metrics["err2"]))
    trial.set_user_attr("precision", np.mean(metrics["precision"]))
    trial.set_user_attr("recall", np.mean(metrics["recall"]))
    trial.set_user_attr("f1", np.mean(metrics["f1"]))
    trial.set_user_attr("logloss", np.mean(metrics["logloss"]))
    
    return np.mean(metrics["auc"])

# Основной блок выполнения
if __name__ == "__main__":
    # Создаем родительский запуск
    with mlflow.start_run(run_name=RUN_NAME) as parent_run:
        parent_run_id = parent_run.info.run_id
        
        # Инициализация MLflow callback с указанием parent run
        mlflc = MLflowCallback(
            tracking_uri=mlflow.get_tracking_uri(),
            metric_name="auc",
            create_experiment=True,
            mlflow_kwargs={
                'experiment_name': EXPERIMENT_NAME,
                'tags': {
                    MLFLOW_PARENT_RUN_ID: parent_run_id,
                    "mlflow.runName": "optuna_trial"
                },
                'nested': True  # Включаем вложенные запуски
            }
        )

        # Создание исследования Optuna
        study = optuna.create_study(
            sampler=optuna.samplers.TPESampler(),
            direction="maximize",
            study_name=STUDY_NAME,
            storage=STUDY_DB_NAME,
            load_if_exists=True
        )
        
        # Запуск оптимизации
        study.optimize(
            objective,
            n_trials=10,
            callbacks=[mlflc],
            show_progress_bar=True
        )

        # Логирование результатов в родительский запуск
        mlflow.log_params(study.best_params)
        mlflow.log_metrics({
            "best_auc": study.best_value,
            "err1": study.best_trial.user_attrs["err1"],
            "err2": study.best_trial.user_attrs["err2"]
        })
        
        # Логирование модели в родительский запуск
        best_model = CatBoostClassifier(**study.best_params, cat_features=cat_features)
        best_model.fit(X_train, y_train, verbose=False)
        
        mlflow.catboost.log_model(
            cb_model=best_model,
            artifact_path="model",
            registered_model_name="best_churn_model",
            pip_requirements=pip_requirements,
            input_example=input_example
        )
    
    # Логирование CatBoost модели


    print("Optimization complete!")

2025-04-06 17:58:39,068 INFO: Tracking URI: http://localhost:5000
/tmp/ipykernel_292372/669051325.py:101: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-04-06 17:58:39,245] Using an existing study with name 'churn_model' instead of creating a new one.


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-04-06 17:58:44,303] Trial 67 finished with value: 0.8381938604812621 and parameters: {'learning_rate': 0.023231873920672563, 'depth': 2, 'l2_leaf_reg': 2.132246311800027, 'random_strength': 0.844876684447144}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 17:58:48,335] Trial 68 finished with value: 0.8413106029093587 and parameters: {'learning_rate': 0.059804189120027385, 'depth': 3, 'l2_leaf_reg': 1.5276612882648632, 'random_strength': 0.7361060281197487}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 17:58:51,915] Trial 69 finished with value: 0.8402809781453962 and parameters: {'learning_rate': 0.057599100615855915, 'depth': 3, 'l2_leaf_reg': 1.2614978128991192, 'random_strength': 0.7253896761120331}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 17:58:56,459] Trial 70 finished with value: 0.841245888982323 and parameters: {'learning_rate': 0.045235616800066714, 'depth': 2, 'l2_leaf_reg': 1.595924738194649, 'random_strength'

/home/mle-user/mle_projects/.venv_name/lib/python3.10/site-packages/mlflow/models/signature.py:333: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  input_schema = _infer_schema(input_ex)
/home/mle-user/mle_projects/.venv_name/lib/python3.10/site-packages/mlflow/models/signature.py:344: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python ca

Optimization complete!


Created version '4' of model 'best_churn_model'.
